* #### DJ Lee


In [ ]:
import copy
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
# from tqdm.notebook import tqdm
# import matplotlib.pyplot as plt
# import seaborn as sns

In [ ]:
# mdl_results_org = pd.read_excel("model_results_1qa_head.xlsx",index_col=0)
mdl_results_org = pd.read_excel("model_results_1qa_tail.xlsx",index_col=0)

In [ ]:
mdl_results_org.tail()

,GDP(SA) True,DFM,BEM1,BEM2,BEM3,LBVAR,TFM,T-M,VAR,SVR-KNN,CRNN,RF-DVS,S-XGB,LQR,OptimARIMA
2015-03-01,0.9,NaN,1.101763,1.330532,NaN,NaN,NaN,NaN,0.895266,1.180455,NaN,0.614753,1.198206,0.115336,0.798744
2015-06-01,0.4,NaN,0.898439,1.233085,NaN,NaN,NaN,NaN,0.887260,0.745444,NaN,0.799948,1.387703,0.159211,0.714233
2015-09-01,1.5,NaN,0.909546,1.230022,NaN,NaN,NaN,NaN,0.932544,1.100864,NaN,0.632852,0.429782,-0.099213,0.486098
2015-12-01,0.8,NaN,1.191104,1.332099,NaN,NaN,NaN,NaN,0.850058,1.066274,NaN,0.830487,0.675963,0.175124,0.530954
2016-03-01,0.3,NaN,0.827477,1.081798,NaN,NaN,NaN,NaN,0.972880,0.766117,NaN,0.744295,0.753825,0.124743,0.343610


In [ ]:
model_names = ['DFM',	'BEM1',	'BEM2',	'BEM3',	#'LBVAR',
               'TFM',	'T-M',	'VAR',	'SVR-KNN',	'CRNN',	'RF-DVS',	'S-XGB',	'LQR',	'OptimARIMA']
# model_names = ['BEM1',	'BEM2', 'VAR', 'SVR-KNN','RF-DVS',	'S-XGB','LQR',	'OptimARIMA']
timeline = mdl_results_org.index

In [ ]:
gdp_realized = mdl_results_org.iloc[0:-6, 0]
mdl_results = mdl_results_org.loc[timeline[0:-6], model_names].copy()

In [ ]:
mdl_results.head()

,BEM1,BEM2,VAR,SVR-KNN,RF-DVS,S-XGB,LQR,OptimARIMA
2007-03-01,0.864427,1.230193,0.994120,0.855742,1.230364,1.387635,0.046405,-0.051158
2007-06-01,1.424863,1.124211,1.034079,1.073497,1.120692,1.300307,0.514080,2.537758
2007-09-01,1.093643,1.091207,1.161066,1.067095,1.235666,1.345291,0.524052,1.354563
2007-12-01,0.960449,0.863250,1.228298,0.837761,1.181119,1.386425,-0.146041,0.461738
2008-03-01,1.177893,0.885476,1.172038,1.186142,0.970269,1.364482,-0.124454,-0.092016


In [ ]:
# 1. Mean, Median, Min, Max
mdl_results['Avg'] = mdl_results.loc[:,model_names].mean(axis=1)
mdl_results['Med'] = mdl_results.loc[:,model_names].median(axis=1)
mdl_results['Min'] = mdl_results.loc[:,model_names].min(axis=1)
mdl_results['Max'] = mdl_results.loc[:,model_names].max(axis=1)

In [ ]:
def AdaMSM_4Q(mdl_results,gdp_realized):
    mdl_results_err = mdl_results.subtract(gdp_realized, axis=0)
    mdl_results_MAE4Q = mdl_results_err.abs().rolling(window=4).mean()
    mdl_results_MAE4Q['Top3Mdls'] = mdl_results_MAE4Q.apply(
        lambda row: tuple(row.nsmallest(3).index), axis=1 )
    mdl_results['AdaMSM'] = np.nan
    top3_lagged = mdl_results_MAE4Q['Top3Mdls'].shift(1)  
    for idx in mdl_results.index:
        if pd.notna(top3_lagged.loc[idx]):
            top3_cols = list(top3_lagged.loc[idx])
            # print(f"{idx}: {mdl_results.loc[idx, top3_cols].median()}")
            mdl_results.loc[idx,'AdaMSM'] = mdl_results.loc[idx, top3_cols].median() + mdl_results_MAE4Q.shift(1).loc[idx,mdl_results.columns[0]]*0 # Add 0 to remove first four results
    return mdl_results.loc[mdl_results.index,'AdaMSM']

In [ ]:
mdl_results['AdaMSM'] = AdaMSM_4Q(mdl_results.loc[:,model_names],gdp_realized)

In [ ]:
def dqs(preds: pd.DataFrame, lamb: float) -> pd.Series:
  taus = getTaus(preds, lamb)
  return pd.Series(map(lambda t, q: preds.loc[t].quantile(q), preds.index, taus), index=preds.index)

def getTaus(preds: pd.DataFrame, lamb: float) -> pd.Series:
  return ((np.sign(preds.mean(axis=1).diff().dropna()) * preds.std(axis=1).diff().dropna().apply(lambda x: max(0, x)) * preds.var(axis=1) * lamb).apply(lambda x: min(max(x, -0.5), 0.5)) + 0.5).fillna(0.5)

In [ ]:
mdl_results['DQS'] = dqs(mdl_results.loc[:,model_names],2.0)

In [ ]:
def dma(mdl_results: pd.DataFrame,gdp_realized,  alpha: float = 1,c: float = 0):
    mdl_results_err = mdl_results.subtract(gdp_realized, axis=0)
    mdl_results_MSE4Q = mdl_results_err.pow(2).rolling(window=4).mean()
    sigma2 = mdl_results_MSE4Q.shift(1).std(axis=1)**2
    likelihood = np.exp(-mdl_results_MSE4Q.shift(1).div(2 * sigma2, axis=0)).div(np.sqrt(2 * np.pi * sigma2),axis=0).pow(alpha) + c
    weights = likelihood.divide(likelihood.sum(axis=1),axis=0)
    return mdl_results.multiply(weights).sum(axis=1)


In [ ]:
mdl_results['BMA'] = dma(mdl_results.loc[:,model_names],gdp_realized)
mdl_results['DMA'] = dma(mdl_results.loc[:,model_names],gdp_realized,.95,1e-3/14)

In [ ]:
def lop(mdl_results: pd.DataFrame,gdp_realized: pd.core.series.Series):
    mdl_results_err = mdl_results.subtract(gdp_realized, axis=0)
    mdl_results_MSE4Q = mdl_results_err.pow(2).rolling(window=4).mean()
    weights=(1/mdl_results_MSE4Q.shift(1)).divide((1/mdl_results_MSE4Q.shift(1)).sum(axis=1),axis=0)
    return mdl_results.multiply(weights).sum(axis=1)


In [ ]:
mdl_results['LOP'] = lop(mdl_results.loc[:,model_names],gdp_realized)

In [ ]:
def AFTER(mdl_results: pd.DataFrame,gdp_realized: pd.core.series.Series):
    mdl_results_err = mdl_results.subtract(gdp_realized, axis=0)
    mdl_results_MSE4Q = mdl_results_err.pow(2).rolling(window=4).mean()
    weights=(1/np.exp(mdl_results_MSE4Q.shift(1))).divide((np.exp(-mdl_results_MSE4Q.shift(1))).sum(axis=1),axis=0)
    return mdl_results.multiply(weights).sum(axis=1)


In [ ]:
mdl_results['AFTER'] = AFTER(mdl_results.loc[:,model_names],gdp_realized)

In [ ]:
mdl_results.tail()

,BEM1,BEM2,VAR,SVR-KNN,RF-DVS,S-XGB,LQR,OptimARIMA,Avg,Med,Min,Max,AdaMSM,DQS,BMA,DMA,LOP,AFTER
2013-09-01,1.334408,1.535399,0.996047,0.981798,0.913084,1.208168,0.104089,0.496023,0.946127,0.988923,0.104089,1.535399,0.913084,0.990855,0.949011,0.948030,0.963955,0.954343
2013-12-01,1.658346,1.473892,0.996946,0.509632,0.845532,1.198780,-0.222024,0.251131,0.839029,0.921239,-0.222024,1.658346,0.996946,0.686798,0.867014,0.868966,0.859926,0.881195
2014-03-01,0.812236,1.252740,0.945466,0.681372,0.879994,0.917285,-0.214069,0.328652,0.700460,0.846115,-0.214069,1.252740,0.879994,0.846115,0.844983,0.843661,0.880295,0.779585
2014-06-01,1.075196,1.309689,0.982197,0.811475,0.800014,1.172090,-0.259720,0.369036,0.782497,0.896836,-0.259720,1.309689,0.811475,0.931975,0.947044,0.945785,0.957106,0.875238
2014-09-01,1.026143,1.271769,0.983936,0.959029,0.741120,0.919554,-0.240744,0.400062,0.757609,0.939291,-0.240744,1.271769,0.959029,0.939291,0.904038,0.903159,0.856786,0.846405


In [ ]:
Ens_methods = ['DQS', 'Avg', 'Med', 'DMA', 'BMA', 'LOP', 'AFTER']

In [ ]:
for col in ['AdaMSM','BMA','DMA','LOP','AFTER']:
    mdl_results.loc[mdl_results.index[0:4], col] = mdl_results.loc[mdl_results.index[0:4], 'Avg'] # 첫 4개 행 비워뒀던 Dynamic 조정하는 녀석들은 Mean으로 채움

In [ ]:
mdl_results.head()

,BEM1,BEM2,VAR,SVR-KNN,RF-DVS,S-XGB,LQR,OptimARIMA,Avg,Med,Min,Max,AdaMSM,DQS,BMA,DMA,LOP,AFTER
2007-03-01,0.864427,1.230193,0.994120,0.855742,1.230364,1.387635,0.046405,-0.051158,0.819716,0.929273,-0.051158,1.387635,0.819716,0.929273,0.819716,0.819716,0.819716,0.819716
2007-06-01,1.424863,1.124211,1.034079,1.073497,1.120692,1.300307,0.514080,2.537758,1.266186,1.122452,0.514080,2.537758,1.266186,1.123064,1.266186,1.266186,1.266186,1.266186
2007-09-01,1.093643,1.091207,1.161066,1.067095,1.235666,1.345291,0.524052,1.354563,1.109073,1.127355,0.524052,1.354563,1.109073,1.127355,1.109073,1.109073,1.109073,1.109073
2007-12-01,0.960449,0.863250,1.228298,0.837761,1.181119,1.386425,-0.146041,0.461738,0.846625,0.911850,-0.146041,1.386425,0.846625,0.856068,0.846625,0.846625,0.846625,0.846625
2008-03-01,1.177893,0.885476,1.172038,1.186142,0.970269,1.364482,-0.124454,-0.092016,0.817479,1.071154,-0.124454,1.364482,1.172038,0.975547,1.028783,1.022258,1.090543,1.049824


In [ ]:
mdl_results.subtract(gdp_realized,axis=0).abs().mean()

BEM1          0.752844
BEM2          0.766712
VAR           0.810532
SVR-KNN       0.658887
RF-DVS        0.510864
S-XGB         0.625815
LQR           1.023561
OptimARIMA    0.818184
Avg           0.538820
Med           0.576177
Min           1.230495
Max           0.916159
AdaMSM        0.576178
DQS           0.681337
BMA           0.558621
DMA           0.558549
LOP           0.552361
AFTER         0.575215
dtype: float64

Data-driven Lambda with new DQS(use IQR instead of standard deviation)

In [ ]:
def dqs_Mk2(preds: pd.DataFrame,gdp_series: pd.Series) -> pd.Series:
  lambdas = get_lambdas(preds,gdp_series)
  taus = getTaus_Mk2(preds, lambdas)
  return pd.Series(map(lambda t, q: preds.loc[t].quantile(q), preds.index, taus), index=preds.index)

def getTaus_Mk2(preds: pd.DataFrame, lamb) -> pd.Series:
  iqr_ = preds.quantile(.75,axis=1)-preds.quantile(.25,axis=1)
  # iqr_ = preds.std(axis=1)
  return ((np.sign(preds.mean(axis=1).diff().dropna()) * (iqr_.diff().dropna().apply(lambda x: max(0, x))) * iqr_.pow(2) * lamb).apply(lambda x: min(max(x, -0.5), 0.5)) + 0.5).fillna(0.5)

def iqr(series):
    return series.quantile(1-.25)-series.quantile(.25)

def get_lambdas(preds, gdp_series):
    history_window = preds.index.values[np.newaxis, :] > gdp_series.index.values[:, np.newaxis]
    iqr_series = pd.Series(
    [gdp_series[history_window[:, i]].quantile(.75) - 
     gdp_series[history_window[:, i]].quantile(.25) 
     for i in range(history_window.shape[1])],
    index=preds.index)
    # iqr_series = pd.Series(
    # [gdp_series[history_window[:, i]].std()
    #  for i in range(history_window.shape[1])],
    # index=preds.index)
    return 1/iqr_series**2

In [ ]:
mdl_results['DQS_Mk2'] = dqs_Mk2(mdl_results,gdp_realized)

In [ ]:
mdl_results.subtract(gdp_realized,axis=0).abs().mean()

BEM1          0.752844
BEM2          0.766712
VAR           0.810532
SVR-KNN       0.658887
RF-DVS        0.510864
S-XGB         0.625815
LQR           1.023561
OptimARIMA    0.818184
Avg           0.538820
Med           0.576177
Min           1.230495
Max           0.916159
AdaMSM        0.576178
DQS           0.681337
BMA           0.558621
DMA           0.558549
LOP           0.552361
AFTER         0.575215
DQS_Mk2       0.611102
dtype: float64

In [ ]:
# mdl_results.to_csv("ens_result_1qa_head.csv",index=True)
mdl_results.to_csv("ens_result_1qa_tail.csv",index=True)

In [ ]:
return

SyntaxError: 'return' outside function (3438313781.py, line 1)

In [ ]:
from scipy import stats

In [ ]:
def get_percentile(row, realized_series):
    actual_val = realized_series[row.name]   
    return stats.percentileofscore(row, actual_val, kind='rank')

Outliers deleted

In [ ]:
def removeOutlier(preds: pd.DataFrame) -> pd.Series:
  preds = copy.deepcopy(preds)
  for t in preds.index:
    lqr = preds.loc[t].quantile(0.75) - preds.loc[t].quantile(0.25)
    lowerBound = preds.loc[t].quantile(0.25) - 1.5 * lqr
    upperBound = preds.loc[t].quantile(0.75) + 1.5 * lqr
    for col in preds.columns:
      if not lowerBound <= preds.loc[t, col] <= upperBound:
        preds.loc[t, col] = np.nan
  return preds


In [ ]:
mdl_results_clean = removeOutlier(mdl_results.loc[:,model_names].copy())
mdl_results_clean.head()

,BEM1,BEM2,SVR-KNN,RF-DVS,S-XGB,LQR,OptimARIMA
2007-03-01,1.048504,1.118599,1.637640,1.380534,1.300307,1.101991,NaN
2007-06-01,1.410123,1.107311,1.303687,1.389925,1.345291,NaN,1.523101
2007-09-01,1.186653,1.143828,1.292385,1.172885,1.386425,0.950896,1.098158
2007-12-01,1.017676,0.928409,1.341312,1.048464,1.364482,NaN,1.445450
2008-03-01,1.194740,1.245658,1.045777,1.556403,1.444494,1.196105,1.061182


In [ ]:
mdl_results_clean['Avg'] = mdl_results_clean.loc[:,model_names].mean(axis=1)
mdl_results_clean['Med'] = mdl_results_clean.loc[:,model_names].median(axis=1)
mdl_results_clean['Max'] = mdl_results_clean.loc[:,model_names].max(axis=1)
mdl_results_clean['Min'] = mdl_results_clean.loc[:,model_names].min(axis=1)
mdl_results_clean['AdaMSM'] = AdaMSM_4Q(mdl_results_clean.loc[:,model_names],gdp_realized)
mdl_results_clean['DQS'] = dqs(mdl_results_clean.loc[:,model_names],2.0)
mdl_results_clean['BMA'] = dma(mdl_results_clean.loc[:,model_names],gdp_realized)
mdl_results_clean['DMA'] = dma(mdl_results_clean.loc[:,model_names],gdp_realized,.95,1e-3/14)

In [ ]:
for col in ['AdaMSM','BMA','DMA','LOP','AFTER']:
    mdl_results_clean.loc[mdl_results_clean.index[0:4], col] = mdl_results_clean.loc[mdl_results_clean.index[0:4], 'Avg'] # 첫 4개 행 비워뒀던 Dynamic 조정하는 녀석들은 Mean으로 채움

In [ ]:
mdl_results_clean.subtract(gdp_realized,axis=0).abs().mean(skipna=False)

BEM1               NaN
BEM2               NaN
SVR-KNN       0.456582
RF-DVS             NaN
S-XGB              NaN
LQR                NaN
OptimARIMA         NaN
Avg           0.435476
Med           0.452931
Max           0.697976
Min           0.514535
AdaMSM             NaN
DQS           0.436302
BMA           0.488298
DMA           0.479496
LOP                NaN
AFTER              NaN
dtype: float64